# 여행 코스 기반 데이터 정리

In [1]:
# 여행객 특성과 방문지 정보 종합 테이블
import numpy as np
import pandas as pd

# CSV 파일 읽기
visit_area_info_df = pd.read_csv('tn_visit_area_info_방문지정보_A.csv')
print('visit_area_info_df:',visit_area_info_df.shape)
traveller_master_df = pd.read_csv('tn_traveller_master_여행객 Master_A.csv')
print('traveller_master_df',traveller_master_df.shape)

# TRAVELER_ID 생성 (TRAVEL_ID 열에서 앞 2자리 제거)
visit_area_info_df['TRAVELER_ID'] = visit_area_info_df['TRAVEL_ID'].astype(str).str[2:]
print('visit_area_info_df',visit_area_info_df.shape)

# DataFrame 조인
merged_df = pd.merge(visit_area_info_df, traveller_master_df, on='TRAVELER_ID', how='inner')
print('merged_df',merged_df.shape)

# # VISIT_AREA_NM 열의 값이 '집'인 행 제거
# filtered_df = merged_df[merged_df['VISIT_AREA_NM'] != '집']
# print('filtered_df',filtered_df.shape)
# filtered_df
filtered_df = merged_df

# filtered_df.to_csv('new_csv/filtered_df.csv', index=False)

visit_area_info_df: (30154, 23)
traveller_master_df (3200, 36)
visit_area_info_df (30154, 24)
merged_df (24288, 59)


In [2]:
# 서울 중구를 한 번이라도 들른 경우에 대한 여행 기록만 남김
result_df = filtered_df[
    filtered_df['ROAD_NM_ADDR'].fillna('').str.contains('서울 중구') |
    filtered_df['LOTNO_ADDR'].fillna('').str.contains('서울 중구')
]
print('result_df',result_df.shape)
# result_df.to_csv('new_csv/result_df.csv', index=False)

result_df (967, 59)


In [3]:
len(np.unique(result_df['TRAVELER_ID']))

348

In [4]:
# 서울 중구를 한 번이라도 들른 경우에 대한 여행 기록
filtered_df_02 = filtered_df[filtered_df['TRAVEL_ID'].isin(np.unique(result_df['TRAVEL_ID']))]
# filtered_df_02.to_csv('new_csv/filtered_df_02.csv', index=False)
print('filtered_df_02',filtered_df_02.shape)

filtered_df_02 (4375, 59)


In [5]:
# # 첫 번째로 들른 장소(집) 제거
# filtered_df_03 = filtered_df_02[filtered_df_02['VISIT_ORDER'] != 1]
# filtered_df_03.to_csv('new_csv/filtered_df_03.csv', index=False)
# print('filtered_df_03',filtered_df_03.shape)
filtered_df_03 = filtered_df_02

In [6]:
# # X_COORD, Y_COORD(위도, 경도) 값이 NULL일 경우 제거 ← 집/거리 등 제거 목적
# filtered_df_04 = filtered_df_03[(filtered_df_03['X_COORD'].isnull() == False) & (filtered_df_03['Y_COORD'].isnull() == False)]
# filtered_df_04.to_csv('new_csv/filtered_df_04.csv', index=False)
# print('filtered_df_04',filtered_df_04.shape)
filtered_df_04 = filtered_df_03

In [7]:
# # VISIT_AREA_NM 기준 아파트/호텔/정류장/터미널 단어 포함일 경우 제거
# filtered_df_05 = filtered_df_04[filtered_df_04['VISIT_AREA_NM'].str.contains('아파트|호텔|정류장|터미널') == False]
# filtered_df_05.to_csv('new_csv/filtered_df_05.csv', index=False)
# print('filtered_df_05',filtered_df_05.shape)
filtered_df_05 = filtered_df_04

In [8]:
# TRAVEL_ID 순으로 정렬하고, 각 TRAVEL_ID 내에서 VISIT_ORDER 순으로 정렬
filtered_df_05_sorted = filtered_df_05.sort_values(by=['TRAVEL_ID', 'VISIT_ORDER'])
# filtered_df_05_sorted.to_csv('new_csv/filtered_df_05_sorted.csv', index=False)
print('filtered_df_05_sorted',filtered_df_05_sorted.shape)

filtered_df_05_sorted (4375, 59)


In [9]:
#여행 코스에 대한 해당 장소 이름 및 위치 정보만 추출
filtered_df_05_sorted_selected = filtered_df_05_sorted[[
    'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'VISIT_START_YMD', 'VISIT_END_YMD', 'ROAD_NM_ADDR', 'LOTNO_ADDR', 'X_COORD', 'Y_COORD',
    'RESIDENCE_TIME_MIN', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'VISIT_CHC_REASON_CD', 'LODGING_TYPE_CD', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION'
]]
# filtered_df_05_sorted_selected.to_csv('new_csv/filtered_df_05_sorted_selected.csv', index=False)
print('filtered_df_05_sorted_selected',filtered_df_05_sorted_selected.shape)

filtered_df_05_sorted_selected (4375, 17)


In [10]:
len(np.unique(filtered_df_05_sorted_selected['TRAVEL_ID']))

348

In [11]:
# len(np.unique(filtered_df_05_sorted_selected['TRAVEL_ID']))

print('코스 내의 장소 전체 가짓수',len(np.unique(filtered_df_05_sorted_selected['VISIT_AREA_NM'])))
filtered_df_05_sorted_selected[filtered_df_05_sorted_selected['ROAD_NM_ADDR'].fillna('').str.contains('서울 중구')]
print('서울 중구 장소 가짓수', len(np.unique(filtered_df_05_sorted_selected[
    filtered_df_05_sorted_selected['ROAD_NM_ADDR'].fillna('').str.contains('서울 중구') | 
    filtered_df_05_sorted_selected['LOTNO_ADDR'].fillna('').str.contains('서울 중구')
]['LOTNO_ADDR'].astype(str))))

코스 내의 장소 전체 가짓수 2214
서울 중구 장소 가짓수 365


## 해당 장소가 뭐하는 곳인지, 여행자가 그 장소에서 무슨 활동을 했는지 정보 보완

In [12]:
# tn_activity_his_활동내역_A.csv
activity_df = pd.read_csv('tn_activity_his_활동내역_A.csv')
print('activity_df', activity_df.shape)

print('activity_df의 방문지 가짓수:', len(np.unique(activity_df['VISIT_AREA_ID'])))

activity_df (34520, 9)
activity_df의 방문지 가짓수: 1402


In [13]:
df = filtered_df_05_sorted_selected.copy()
tc_codeb = pd.read_csv('tc_codeb_코드B.csv')

# - RESIDENCE_TIME_MIN
#     - 값 → ~분
# df['RESIDENCE_TIME_MIN'] = df['RESIDENCE_TIME_MIN'].astype(str) + '분 동안 체류함.'
df['RESIDENCE_TIME_MIN'] = df['RESIDENCE_TIME_MIN']

# - VISIT_AREA_TYPE_CD
#     - 타테이블 내에 정보 위치
# VISIT_AREA_TYPE_CD 열 값을 대체
df['VISIT_AREA_TYPE_CD'] = df['VISIT_AREA_TYPE_CD'].astype(str)
tc_codeb['cd_b'] = tc_codeb['cd_b'].astype(str)

tc_codeb_VISIT_AREA_TYPE_CD = tc_codeb[tc_codeb['cd_a']=='VIS']
df = pd.merge(df, tc_codeb_VISIT_AREA_TYPE_CD[['cd_b', 'cd_nm']], how='left', left_on='VISIT_AREA_TYPE_CD', right_on='cd_b')
df['VISIT_AREA_TYPE_CD'] = df['cd_nm']
df = df.drop(columns=['cd_b', 'cd_nm'])
df

# - REVISIT_YN
#     - 값 → 재방문 여부
df.loc[df['REVISIT_YN'] == 'Y', 'REVISIT_YN'] = "재방문 했음."
df.loc[df['REVISIT_YN'] == 'N', 'REVISIT_YN'] = "재방문 안 했음."

# - VISIT_CHC_REASON_CD
#     - 타테이블 내에 정보 위치
df['VISIT_CHC_REASON_CD'] = df['VISIT_CHC_REASON_CD'].fillna(0).astype(int).astype(str)
tc_codeb['cd_b'] = tc_codeb['cd_b'].astype(str)

tc_codeb_VISIT_CHC_REASON_CD = tc_codeb[tc_codeb['cd_a']=='REN']
df = pd.merge(df, tc_codeb_VISIT_CHC_REASON_CD[['cd_b', 'cd_nm']], how='left', left_on='VISIT_CHC_REASON_CD', right_on='cd_b')

df['VISIT_CHC_REASON_CD'] = df['cd_nm'].fillna('')
df = df.drop(columns=['cd_b', 'cd_nm'])
df

# - LODGING_TYPE_CD
#     - 타테이블 내에 정보 위치
df['LODGING_TYPE_CD'] = df['LODGING_TYPE_CD'].fillna(0).astype(int).astype(str)
tc_codeb['cd_b'] = tc_codeb['cd_b'].astype(str)

tc_codeb_LODGING_TYPE_CD = tc_codeb[tc_codeb['cd_a']=='HTY']
df = pd.merge(df, tc_codeb_LODGING_TYPE_CD[['cd_b', 'cd_nm']], how='left', left_on='LODGING_TYPE_CD', right_on='cd_b')

df['LODGING_TYPE_CD'] = df['cd_nm'].fillna('')
df = df.drop(columns=['cd_b', 'cd_nm'])
df

# - DGSTFN
#     - 타테이블 내에 정보 위치
df['DGSTFN'] = df['DGSTFN'].fillna(0).astype(int).astype(str)
tc_codeb['cd_b'] = tc_codeb['cd_b'].astype(str)

tc_codeb_DGSTFN = tc_codeb[tc_codeb['cd_a']=='DGS']
df = pd.merge(df, tc_codeb_DGSTFN[['cd_b', 'cd_nm']], how='left', left_on='DGSTFN', right_on='cd_b')

df['DGSTFN'] = df['cd_nm'].fillna('')
df = df.drop(columns=['cd_b', 'cd_nm'])
df

# - REVISIT_INTENTION
#     - 타테이블 내에 정보 위치
df['REVISIT_INTENTION'] = df['REVISIT_INTENTION'].fillna(0).astype(int).astype(str)
tc_codeb['cd_b'] = tc_codeb['cd_b'].astype(str)

tc_codeb_REVISIT_INTENTION = tc_codeb[tc_codeb['cd_a']=='REP']
df = pd.merge(df, tc_codeb_REVISIT_INTENTION[['cd_b', 'cd_nm']], how='left', left_on='REVISIT_INTENTION', right_on='cd_b')

df['REVISIT_INTENTION'] = df['cd_nm'].fillna('')
df = df.drop(columns=['cd_b', 'cd_nm'])
df

# - RCMDTN_INTENTION
#     - 타테이블 내에 정보 위치
df['RCMDTN_INTENTION'] = df['RCMDTN_INTENTION'].fillna(0).astype(int).astype(str)
tc_codeb['cd_b'] = tc_codeb['cd_b'].astype(str)

tc_codeb_RCMDTN_INTENTION = tc_codeb[tc_codeb['cd_a']=='REC']
df = pd.merge(df, tc_codeb_RCMDTN_INTENTION[['cd_b', 'cd_nm']], how='left', left_on='RCMDTN_INTENTION', right_on='cd_b')

df['RCMDTN_INTENTION'] = df['cd_nm'].fillna('')
df = df.drop(columns=['cd_b', 'cd_nm'])
df

# df.to_csv('new_csv/df.csv', index=False)
print('df',df.shape)

df (4375, 17)


In [14]:
# 불필요한 장소 정보 제거
mo_df = df[df['VISIT_AREA_TYPE_CD'].isin(['사무실', '역, 터미널, 고속도로 휴게소', '집', '친구/친지집']) != True]
mo_df.shape

# mo_df.to_csv('new_csv/mo_df.csv', index=False)
print('mo_df',mo_df.shape)

mo_df (3228, 17)


In [15]:
# 서울 중구를 한 번이라도 들른 경우에 대한 여행 기록
now_df = mo_df.copy()
print('mo_df',mo_df.shape)

now_df = now_df[now_df['TRAVEL_ID'].isin(np.unique(now_df[
    now_df['ROAD_NM_ADDR'].fillna('').str.contains('서울 중구') |
    now_df['LOTNO_ADDR'].fillna('').str.contains('서울 중구')
]['TRAVEL_ID']))]
# now_df.to_csv('new_csv/now_df.csv', index=False)
print('now_df',now_df.shape)

mo_df (3228, 17)
now_df (3143, 17)


In [16]:
# # 선별

# temp_df = df[df['VISIT_AREA_TYPE_CD'].isin(['사무실', '역, 터미널, 고속도로 휴게소', '집', '친구/친지집'])]
# unique_values_counts_df = pd.DataFrame(np.unique(temp_df['VISIT_AREA_TYPE_CD'], return_counts=True)).transpose()
# unique_values_counts_df.columns = ['VISIT_AREA_TYPE_CD', 'Count']
# unique_values_counts_df

In [17]:
# X_COORD, Y_COORD(위도, 경도) 값이 NULL일 경우 제거 ← 집/거리 등 제거 목적
now_03_df = now_df[(now_df['X_COORD'].isnull() == False) & (now_df['Y_COORD'].isnull() == False)]
# now_03_df.to_csv('new_csv/now_03_df.csv', index=False)
print('now_03_df',now_03_df.shape)

now_03_df (3044, 17)


In [18]:
# 현황
now_state_df = now_03_df
print('여행 코스 가짓수', len(np.unique(now_state_df['TRAVEL_ID'])))
print('전체 장소 가짓수', len(np.unique(now_state_df['VISIT_AREA_NM'])))
print('중구 장소 가짓수', len(np.unique(now_state_df[
    now_state_df['ROAD_NM_ADDR'].fillna('').str.contains('서울 중구') |
    now_state_df['LOTNO_ADDR'].fillna('').str.contains('서울 중구')
]['VISIT_AREA_NM'])))

여행 코스 가짓수 338
전체 장소 가짓수 1764
중구 장소 가짓수 426


In [22]:
# 일반 소비 내역 붙이기
import pandas as pd

# df_2 CSV 파일 불러오기
df_2 = pd.read_csv("tn_activity_consume_his_활동소비내역_A.csv")

# 두 데이터프레임을 TRAVEL_ID와 ROAD_NM_ADDR 컬럼을 기준으로 병합
now_04_df = now_03_df.merge(df_2[['TRAVEL_ID', 'ROAD_NM_ADDR', 'PAYMENT_AMT_WON', 'PAYMENT_ETC']],
            on=['TRAVEL_ID', 'ROAD_NM_ADDR'], how='left')

In [23]:
now_04_df

,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,PAYMENT_AMT_WON,PAYMENT_ETC
0,a_a000029,3,아이센스리그PC방 압구정점,2022-08-13,2022-08-13,서울 강남구 압구정로 326,서울 강남구 신사동 660-2,127.037167,37.528348,60.0,기타,재방문 안 했음.,기타,,보통,재방문할 의향 보통임.,추천 의향 없음.,NaN,NaN
1,a_a000029,4,키친마이야르,2022-08-13,2022-08-13,서울 강남구 언주로170길 22,서울 강남구 신사동 644-6,127.035423,37.526609,30.0,식당/카페,재방문 안 했음.,지명도/명소/핫플레이스,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,46000.0,마이야르 스테이크_L;오딸바 크리스탈 에이
2,a_a000029,6,3게스트하우스,2022-08-13,2022-08-13,서울 중구 소공로6길 34,서울 중구 남산동2가 43-8,126.984993,37.558437,120.0,숙소,재방문 안 했음.,가기 편해서/교통이 좋아서,게스트하우스,만족,재방문할 의향 보통임.,추천 의향 있음.,NaN,NaN
3,a_a000029,7,하남돼지집 대학로점,2022-08-13,2022-08-13,서울 종로구 대학로8가길 52,서울 종로구 동숭동 1-48,127.002640,37.582995,90.0,식당/카페,재방문 안 했음.,지인의 추천이 있어서,,만족,재방문할 의향 보통임.,추천 의향 있음.,NaN,NaN
4,a_a000029,9,3게스트하우스,2022-08-13,2022-08-14,서울 중구 소공로6길 34,서울 중구 남산동2가 43-8,126.984993,37.558437,30.0,숙소,재방문 했음.,가기 편해서/교통이 좋아서,게스트하우스,만족,재방문할 의향 보통임.,추천 의향 있음.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3483,d_d005429,9,이태원거리,2022-10-29,2022-10-29,NaN,서울 용산구 이태원동 119-23,126.995809,37.534561,240.0,"상업지구(거리, 시장, 쇼핑시설)",재방문 안 했음.,지명도/명소/핫플레이스,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,NaN,NaN
3484,d_d005429,10,남산서울타워,2022-10-29,2022-10-29,서울특별시 중구 남산공원길 125-72,서울 용산구 용산동2가 산 1-3,126.988217,37.551280,90.0,"상업지구(거리, 시장, 쇼핑시설)",재방문 안 했음.,지명도/명소/핫플레이스,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,NaN,NaN
3485,d_d005429,12,청와대 정문,2022-10-30,2022-10-30,NaN,서울 종로구 세종로 1,126.975240,37.584368,150.0,"역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등)",재방문 안 했음.,지명도/명소/핫플레이스,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,NaN,NaN
3486,d_d005429,13,토속촌삼계탕,2022-10-30,2022-10-30,서울 종로구 자하문로5길 5,서울 종로구 체부동 85-1,126.971495,37.577723,60.0,식당/카페,재방문 안 했음.,지나가다 우연히,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,NaN,NaN


In [24]:
# 숙소 소비 내역 붙이기

# df_3도 CSV 파일로 불러오기
df_3 = pd.read_csv("tn_lodge_consume_his_숙박소비내역_A.csv")

# df에 NaN 값이 있는 PAYMENT_AMT_WON와 PAYMENT_ETC 열을 df_3의 값으로 병합
now_05_df = now_04_df.merge(df_3[['TRAVEL_ID', 'ROAD_NM_ADDR', 'PAYMENT_AMT_WON', 'PAYMENT_ETC']],
            on=['TRAVEL_ID', 'ROAD_NM_ADDR'], how='left', suffixes=('', '_df3'))

# NaN 값을 df_3의 값으로 채우기
now_05_df['PAYMENT_AMT_WON'] = now_05_df['PAYMENT_AMT_WON'].fillna(now_05_df['PAYMENT_AMT_WON_df3'])
now_05_df['PAYMENT_ETC'] = now_05_df['PAYMENT_ETC'].fillna(now_05_df['PAYMENT_ETC_df3'])

# 병합 시 추가된 임시 컬럼 제거
now_05_df.drop(columns=['PAYMENT_AMT_WON_df3', 'PAYMENT_ETC_df3'], inplace=True)

# 결과 확인
now_05_df


,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,PAYMENT_AMT_WON,PAYMENT_ETC
0,a_a000029,3,아이센스리그PC방 압구정점,2022-08-13,2022-08-13,서울 강남구 압구정로 326,서울 강남구 신사동 660-2,127.037167,37.528348,60.0,기타,재방문 안 했음.,기타,,보통,재방문할 의향 보통임.,추천 의향 없음.,NaN,NaN
1,a_a000029,4,키친마이야르,2022-08-13,2022-08-13,서울 강남구 언주로170길 22,서울 강남구 신사동 644-6,127.035423,37.526609,30.0,식당/카페,재방문 안 했음.,지명도/명소/핫플레이스,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,46000.0,마이야르 스테이크_L;오딸바 크리스탈 에이
2,a_a000029,6,3게스트하우스,2022-08-13,2022-08-13,서울 중구 소공로6길 34,서울 중구 남산동2가 43-8,126.984993,37.558437,120.0,숙소,재방문 안 했음.,가기 편해서/교통이 좋아서,게스트하우스,만족,재방문할 의향 보통임.,추천 의향 있음.,60000.0,NaN
3,a_a000029,7,하남돼지집 대학로점,2022-08-13,2022-08-13,서울 종로구 대학로8가길 52,서울 종로구 동숭동 1-48,127.002640,37.582995,90.0,식당/카페,재방문 안 했음.,지인의 추천이 있어서,,만족,재방문할 의향 보통임.,추천 의향 있음.,NaN,NaN
4,a_a000029,9,3게스트하우스,2022-08-13,2022-08-14,서울 중구 소공로6길 34,서울 중구 남산동2가 43-8,126.984993,37.558437,30.0,숙소,재방문 했음.,가기 편해서/교통이 좋아서,게스트하우스,만족,재방문할 의향 보통임.,추천 의향 있음.,60000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3511,d_d005429,9,이태원거리,2022-10-29,2022-10-29,NaN,서울 용산구 이태원동 119-23,126.995809,37.534561,240.0,"상업지구(거리, 시장, 쇼핑시설)",재방문 안 했음.,지명도/명소/핫플레이스,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,NaN,NaN
3512,d_d005429,10,남산서울타워,2022-10-29,2022-10-29,서울특별시 중구 남산공원길 125-72,서울 용산구 용산동2가 산 1-3,126.988217,37.551280,90.0,"상업지구(거리, 시장, 쇼핑시설)",재방문 안 했음.,지명도/명소/핫플레이스,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,NaN,NaN
3513,d_d005429,12,청와대 정문,2022-10-30,2022-10-30,NaN,서울 종로구 세종로 1,126.975240,37.584368,150.0,"역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등)",재방문 안 했음.,지명도/명소/핫플레이스,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,NaN,NaN
3514,d_d005429,13,토속촌삼계탕,2022-10-30,2022-10-30,서울 종로구 자하문로5길 5,서울 종로구 체부동 85-1,126.971495,37.577723,60.0,식당/카페,재방문 안 했음.,지나가다 우연히,,매우만족,재방문할 의향 매우 있음.,추천 의향 매우 있음.,NaN,NaN


In [25]:
now_05_df.to_csv('new_csv/now_05_df.csv', index=False)